# Basic setup for all notebooks

Import commonly-used packages, setup for making maps, setup file structure, read in some basic buoy information.

This notebook is read in by other notebooks; it is not meant to be run alone.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('max_rows', 6)  # max number of rows to show in this notebook — to save space!
from matplotlib.dates import date2num
import numpy as np
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean.cm as cmo
import os
from collections import OrderedDict
import matplotlib as mpl
mpl.rcParams['font.size'] = 14
from scipy.optimize import curve_fit
import shapely
import shapely.ops
import tabs
import xarray as xr
from shutil import unpack_archive
from glob import glob
from scipy.stats import expon, norm
from ast import literal_eval
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
land_10m = cartopy.feature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='face',
                                        facecolor='0.8')
coast_10m = cartopy.feature.NaturalEarthFeature('physical', 'coastline', '10m',
                                        edgecolor='k',
                                        facecolor='none')
landwater_10m = cartopy.feature.NaturalEarthFeature('physical', 'rivers_north_america', '10m',
                                        edgecolor='k',
                                        facecolor='none')
states_provinces = cartopy.feature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    facecolor='none')
pc = cartopy.crs.PlateCarree()
merc = cartopy.crs.Mercator(central_longitude=-85.0)
aea = cartopy.crs.AlbersEqualArea(central_longitude=-96.0, standard_parallels=(29.5, 45.5), central_latitude=23.0)
stereo = cartopy.crs.Stereographic(central_latitude=90, central_longitude=-105, true_scale_latitude=60)
# ae = cartopy.crs.AzimuthalEquidistant(central_longitude=-94.9, central_latitude=29.5)

watercolor = '#512DA8'
props = dict(boxstyle='round', facecolor='w', edgecolor='w', alpha=0.8)

b = '../data/'
baseb = b + 'bathy/'
basesf = b + 'shapefiles/'
based = b + 'stations/'
baset = b + 'twdb/'
baser = b + 'rain/'
basell = '%s/saved/' % basesf

# create directories
os.makedirs('../figures', exist_ok=True)
os.makedirs(baseb, exist_ok=True)
os.makedirs(baser, exist_ok=True)
os.makedirs(basell, exist_ok=True)

In [ ]:
date_ssh = pd.Timestamp('2017-8-24 16:00')  # when sea surface starts to raise
date_outflow = pd.Timestamp('2017-8-26 10:00')  # when salinity starts to drop, based on outflow calculation
date_endoutflow = pd.Timestamp('2017-9-8 19:00')

In [ ]:
run buoy_info.py

In [ ]:
# TWDB watershed basin ids for Galveston Bay
idgb = [10120, 10030, 10111, 10101, 10040, 10100, 10110, 10020, 8110, 10081, 10091, 10074, 10061, 10062, 11021, 
       11092, 11081, 11080, 11070, 11094, 24240, 24245, 24320, 11110, 11124, 11122, 24390, 24230, 24210, 24220,
       11030, 11040, 11010, 11003, 24250, 11150, 11130, 11020, 10066, 10050, 24260, 10060, 10075, 10073, 10064,
       10065, 10063, 10090, 10080, 9030, 8010, 8020, 10010, 9010, 10002, 7050, 7070, 7060, 24235]

Read in TWDB subbasins.

In [ ]:
# TWDB river basins
twdbshapes = 'galv_ws_nad83_prj_albers_equal_area_conic_USGS'
fname = '%s%s/%s.shp' % (basesf, twdbshapes, twdbshapes)
reader = cartopy.io.shapereader.Reader(fname)
basins = reader.records()
shapes_twdbbasins = []  # basins in TWDB inflow estimate
shapes_lkhouston = []
shapes_bay = []
shapes_ungauged = []

for basin in basins:  # aea projection
       
    # save all basins in the inflow estimate
    shapes_twdbbasins.append(basin)
    
    wstype = basin.attributes['WS_type']

    # separate out Lake Houston drainage basins
    if "(Lake Houston)" in wstype: 
        shapes_lkhouston.append(basin)
    elif wstype == 'Bay watershed':
        shapes_bay.append(basin)
    elif ("Ungauged" in wstype) or ("Ungaged" in wstype):
        shapes_ungauged.append(basin)

# outline of twdb basins
shapes_twdbbasins_full_aea = shapely.ops.unary_union([shape.geometry for shape in shapes_twdbbasins])

# Shape of Lake Houston drainage
shapes_lkhouston_full_aea = shapely.ops.unary_union([shape.geometry for shape in shapes_lkhouston])

# Shape of Galveston Bay itself
shapes_bay_full_aea = shapely.ops.unary_union([shape.geometry for shape in shapes_bay])

# Shape of Ungauged modeled region
shapes_ungauged_full_aea = shapely.ops.unary_union([shape.geometry for shape in shapes_ungauged])

Read in points making up shapes if they exist. They will exist after `00_setup_shapes` notebook is run.

In [ ]:
fname = '%s/ll_shapes_gbay_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_gbay_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))
    
fname = '%s/ll_shapes_trinity_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_trinity_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))
    
fname = '%s/ll_shapes_twdbbasins_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_twdbbasins_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))
    
fname = '%s/ll_shapes_lkhouston_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_lkhouston_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))
    
fname = '%s/ll_shapes_ungauged_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_ungauged_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))
    
fname = '%s/ll_shapes_west_full.txt' % basell
if os.path.exists(fname):
    ll = np.loadtxt(fname)
    shapes_west_full = shapely.geometry.Polygon(list(zip(ll[:,0], ll[:,1])))

Read in bathymetry data if it exists. It will exist after `0_process_bathy` notebook is run.

In [ ]:
fname = '%sbathy.nc' % baseb
if os.path.exists(fname):
    bathy = xr.open_dataset(fname)

Read in station time series data if it exists. It will exist after `1_gather_data` notebook is run.

In [ ]:
fname = '%sdata.csv' % based
# fname = '/Volumes/GoogleDrive/My Drive/Documents/papers/inprogress/harvey_inflow/old/old/code/data/stations/data.csv'
# fname = '/Volumes/GoogleDrive/My Drive/Documents/papers/inprogress/harvey_inflow/old/old/code/data/oldstations/data2017-15min.csv'
if os.path.exists(fname):
    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    dt = (df.index[1] - df.index[0]).total_seconds()  # time step in seconds
    dates = df.index

Read in rain indices for regions. This will exist after `1_gather_data` notebook is run.

In [ ]:
fname = '%s/radar_rain_combined.nc' % baser
if os.path.exists(fname):
    rain = xr.open_dataset(fname)

Read in rain indices for regions. This will exist after `1_gather_data` notebook is run.

In [ ]:
fname = '%s/radar.csv' % baser
if os.path.exists(fname):
    radar = pd.read_csv(fname, index_col=0)
    # literal_eval allows reading in lists from csv file
    radar['irain'] = radar['irain'].apply(literal_eval)
    radar['rain [m^3]'] = radar['rain [m^3]'].apply(literal_eval)